# Defect map example 

In [1]:
import numpy as np

In [2]:
Nx = 10
Nz = 20
dx = 0.5 #[mm]
dz = 0.25 #[mm]
p_def = np.array([4.2*dx, 10.7*dz])
p_def_hat = np.around(p_def)
print(p_def)
print(p_def_hat)

[2.1   2.675]
[2. 3.]


In [7]:
print(np.floor(p_def/np.array([dx, dz])))
print(np.ceil(p_def/np.array([dx, dz])))

[ 4. 10.]
[ 5. 11.]


In [8]:
"""
p_def is somewhere within p1, p2, p3 and p4
p1 = (n, m), p2 = (n, m+1), p3 = (n+1, m), p4 = (n+1, m+1)
"""
p_floored = np.floor(p_def/np.array([dx, dz])).astype(int)
n = p_floored[0]
m = p_floored[1]
print(n)
print(m)

4
10


In [5]:
def get_distance(p_def, p):
    return np.sqrt((p_def[0] - p[0])**2 + (p_def[1] - p[1])**2)

# 2D case
neighbors_idx = np.array([[n, m], [n, m+1], [n+1, m], [n+1, m+1]])
neighbors = neighbors_idx.astype(float)
neighbors[:, 0] = neighbors[:, 0]* dx
neighbors[:, 1] = neighbors[:, 1]* dz
distance = np.zeros(neighbors.shape[0])

for idx in range(4):
    distance[idx] = get_distance(p_def, neighbors[idx])
# Normalize teh distance w/ the total distance
distance_norm = distance/ np.sum(distance)
energy_scaled = 1/distance_norm
# Normalize the energy
energy = energy_scaled/np.sum(energy_scaled)
energy

array([0.28016261, 0.45174864, 0.12933526, 0.13875349])

In [6]:
def_map = np.zeros(Nx*Nz)
# Allocate the corresponding energy to the neighboring points
for count, p in enumerate(neighbors_idx):
    # Convert 2D -> 1D 
    idx = p[0]*Nx + p[1]
    def_map[idx] = energy[count]
nz = np.nonzero(def_map)[0]
print(def_map[nz] == energy)

[ True  True  True  True]


array([0.28016261, 0.45174864, 0.12933526, 0.13875349])

In [3]:
class DefectMapSingleDefect():
    """ Class to generate a defect map for FWM. 
    When a defect is located not exactly on a measurement grid point, rather between grid points, we usually round 
    the defect position to the nearest grid point to obtaine an "binary" defect map", where the quantization error is 
    inevitable. 
    This class aims to minimize such quantization error by representing the defect map with the "energy" 
    of the position. The energy is inversely proportional to the distance b/w the defect and a neighboring grid point.
    
    (***) Up to now, this class considers only a single defect case. When there are multiple defects in a test object,
    iterate over the defect position and the resulting defect map is the sum of all defect maps. 
    
    TODO: write a unittest
    
    """
    
    def __init__(self):
        self.def_map = None
        self.energy = None
        self.p_def = None
    
    def calculate_distance(self, p):
        return np.sqrt((self.p_def[0] - p[0])**2 + (self.p_def[1] - p[1])**2)
    
    def get_defect_map(self):
        return self.def_map
    
    def test_def_map(self):
        nz = np.nonzero(self.def_map)[0]
        print(self.def_map[nz] == self.energy)
 

class DefectMapSingleDefect2D(DefectMapSingleDefect):
    
    def __init__(self, p_def, Nx, Nz, dx, dz):
        self.p_def = np.array(p_def)
        self.Nx = int(Nx)
        self.Nz = int(Nz)
        self.dx = float(dx)
        self.dz = float(dz)
        
    def find_neighbors(self):
        # Index for the neighbors
        p_floored = np.floor(p_def/np.array([dx, dz])).astype(int)
        n = p_floored[0]
        m = p_floored[1]
        self.neighbors_idx = np.array([[n, m], [n, m+1], [n+1, m], [n+1, m+1]])
        # Correct position of the neighbors
        self.neighbors = self.neighbors_idx.astype(float)
        self.neighbors[:, 0] = self.neighbors[:, 0]* self.dx
        self.neighbors[:, 1] = self.neighbors[:, 1]* self.dz
        
    def calculate_energy(self): 
        """ Allocate the "energy" of the defect according to the distance b/w the defect and each neighboring point 
        """
        distance = np.zeros(self.neighbors.shape[0])
        for idx in range(4):
            distance[idx] = self.calculate_distance(self.neighbors[idx])
        # Normalize teh distance w/ the total distance
        distance_norm = distance/ np.sum(distance)
        energy_scaled = 1/distance_norm
        # Normalize the energy
        self.energy = energy_scaled/np.sum(energy_scaled)
        
    def generate_defect_map(self):
        """
        Test
        ----
        nz = np.nonzero(self.def_map)[0]
        print(self.def_map[nz] == self.energy)
        """
        self.find_neighbors()
        self.calculate_energy()
        self.def_map = np.zeros(Nx*Nz)
        # Allocate the corresponding energy to the neighboring points
        for count, p in enumerate(self.neighbors_idx):
            # Convert 2D -> 1D 
            idx = p[0]* self.Nx + p[1]
            self.def_map[idx] = self.energy[count]      
    

In [4]:
dm2d = DefectMapSingleDefect2D(p_def, Nx, Nz, dx, dz)
dm2d.generate_defect_map()
def_map2 = dm2d.get_defect_map()
dm2d.test_def_map()
nz2 = np.nonzero(def_map2)[0]
print(def_map2[nz2])

[ True  True  True  True]
[0.28016261 0.45174864 0.12933526 0.13875349]
